In [1]:
#import pandas as pd
import numpy as np
import sys
from multiprocessing import Process, Pipe
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from os import cpu_count

In [2]:
try:
    with open('./input/inputs.npy',"rb") as f:
        inputs = np.load(f)
    with open('./input/outputs.npy',"rb") as f:
        outputs = np.load(f)
        
except:
    data = np.genfromtxt('./input/train.csv', delimiter=',',skip_header=1,dtype=np.uint8)
    inputs = data[:,1:]
    outputs = data[:,0]
    np.save('./input/inputs.npy',inputs)
    np.save('./input/outputs.npy',outputs)
print(f'Input:\nShape:{inputs.shape}\n{inputs[:5,:5]}\n')
print(f'Output:\nShape:{outputs.shape}\n{outputs}\n')

Input:
Shape:(42000, 784)
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Output:
Shape:(42000,)
[1 0 1 ... 7 6 9]



In [3]:
def onehot(ARRAY):
    return np.eye(10)[ARRAY].astype(np.uint8).T

In [16]:
inputs = normalize(inputs)
X_train, X_test, y_train, y_test = train_test_split(inputs,outputs,test_size = 0.95)

X_train = X_train.T
y_train_hot = onehot(y_train)

l_dim = np.array([X_train.shape[0],485,299,185,114,71,44,27,17,10], dtype = np.uint16)
w = [int]*(len(l_dim)+1)
b = [int]*(len(l_dim)+1)

In [17]:
G = X_train.shape[0]
g = []
phi = (1 + np.sqrt(5))/2
i = 0
while G >= 10:
    print(G)
    G = int(round(G/phi,0))


784
485
300
185
114
70
43
27
17
11


In [28]:
def sigmoid(X):
    return np.exp(X)/(1 + np.exp(X))

def sigprime(X):
    return np.exp(X)/np.power(1 + np.exp(X),2)

def initialize_wb(LAYERS):
    for i in range(0,len(LAYERS)-1):
        w[i] = np.array(np.random.random((LAYERS[i+1],LAYERS[i])), dtype=np.float128)
        b[i]  = np.array(np.random.random((LAYERS[i+1],1)), dtype=np.float128)
    return w, b

def feedforward(W,X,B):
    return normalize(sigmoid(np.dot(W,X) + B))

def costfunction(PRED,TRUE):
    return np.sum(np.power(np.subtract(PRED,TRUE),2))

def backprop(WA, WB, AA, AB, AB_TRUE, BA): #Layer A calculates Layer B, we update weights in layer A
    # WA - "weight layer A"
    # AB - "activation layer B"
    # BA - "bias layer A"   
    w_adj = 1/len(AB)*2*np.sum((AB-AB_TRUE)*sigprime(AB))*AA
    b_adj = 2*np.sum((AB-AB_TRUE)*sigprime(AB))
    w_new = WA
    for i in range(len(WA)):
        w_new[i] = WA[i] - np.average(w_adj,axis=1)[i]
    b_new = BA
    for i in range(len(BA)):
        b_new[i] = BA[i] - b_adj
    return w_new, b_new

In [ ]:
backprop(w[7],w[8],a[7],a[8],y_train,b[8])

In [61]:
w, b = initialize_wb(l_dim)
a = [] #activations
A = a #recalculated activations
layer = X_train
for i in range(0,len(l_dim)-1):
    sys.stdout.write(f'{i}\r')
    newlayer = feedforward(w[i],layer,b[i])
    print(newlayer.shape)
    a.append(newlayer)
    layer = newlayer
for i in reversed(range(0,len(l_dim)-1)):
    w[i], b[i] = backprop(w[i-1],w[i],a[i-1],a[i],)

(485, 2100)
(299, 2100)
(185, 2100)
(114, 2100)
(71, 2100)
(44, 2100)
(27, 2100)
(17, 2100)
(10, 2100)


TypeError: backprop() missing 2 required positional arguments: 'AB_TRUE' and 'BA'

In [37]:
print(f'{layer[9].shape}')
print(f'{y_train.shape}')

(2100,)
(2100,)


In [14]:
for l in layers:
    print(l.shape)

(485, 2100)
(299, 2100)
(185, 2100)
(114, 2100)
(71, 2100)
(44, 2100)
(27, 2100)
(17, 2100)
(10, 2100)


In [37]:
layer[8].shape

(2100,)

In [17]:
backprop(w[])

[array([[0.02188446, 0.02186296, 0.02197822, ..., 0.02198359, 0.02190449,
         0.02165446],
        [0.02187859, 0.02184836, 0.02192142, ..., 0.02191524, 0.02189598,
         0.02168718],
        [0.02175002, 0.02175859, 0.02199114, ..., 0.02199479, 0.02186999,
         0.02162903],
        ...,
        [0.0218813 , 0.02185595, 0.02195276, ..., 0.02194159, 0.02187636,
         0.02169529],
        [0.02178212, 0.02175541, 0.02193507, ..., 0.02194156, 0.02190111,
         0.02165755],
        [0.021805  , 0.02173554, 0.02190916, ..., 0.02189457, 0.02188438,
         0.02179981]]),
 array([[0.02182216, 0.02182137, 0.02182489, ..., 0.02182486, 0.02182368,
         0.02181951],
        [0.02182212, 0.02182142, 0.02182478, ..., 0.02182473, 0.02182359,
         0.02181959],
        [0.02182216, 0.02182142, 0.02182503, ..., 0.021825  , 0.02182372,
         0.02181933],
        ...,
        [0.0218221 , 0.02182149, 0.02182433, ..., 0.0218243 , 0.02182331,
         0.02181989],
        [0.0